In [248]:
pip install dash==2.8.1

  Using cached dash-2.8.1-py3-none-any.whl.metadata (11 kB)
Using cached dash-2.8.1-py3-none-any.whl (9.9 MB)
  Attempting uninstall: dash
    Found existing installation: dash 3.1.1
    Uninstalling dash-3.1.1:
      Successfully uninstalled dash-3.1.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash-bootstrap-components 2.0.3 requires dash>=3.0.4, but you have dash 2.8.1 which is incompatible.


In [249]:
!pip install jupyter-dash



In [250]:
data = pd.read_csv("avocado.csv")
data

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,7,2018-02-04,1.63,17074.83,2046.96,1529.20,0.00,13498.67,13066.82,431.85,0.0,organic,2018,WestTexNewMexico
18245,8,2018-01-28,1.71,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,2018,WestTexNewMexico
18246,9,2018-01-21,1.87,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,2018,WestTexNewMexico
18247,10,2018-01-14,1.93,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,2018,WestTexNewMexico


In [251]:
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output


# Load the data
data = pd.read_csv("avocado.csv")
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")

# Get unique regions and types
regions = sorted(data["region"].unique())
avocado_types = data["type"].unique()

In [252]:
# Third cell - create app with external CSS
app = JupyterDash(__name__, title="Avocado Dashboard", external_stylesheets=['style.css'])

app.layout = html.Div([
    # Header with title
    html.Div(
        children=[
            html.P(children="🥑", className="header-emoji"),
            html.H1(
                children="Avocado Analytics", className="header-title"
            ),
            html.P(
                children=(
                    "Analyze the behavior of avocado prices and the number"
                    " of avocados sold in the US between 2015 and 2018"
                ),
                className="dash-header__subtitle"
            )
        ], 
        className="dash-header"
    ),
    
    # Menu bar with filters
    html.Div([
        # Region filter
        html.Div([
            html.Div("SELECT REGION", className="menu-title"),
            dcc.Dropdown(
                id="region-filter",
                options=[{"label": r, "value": r} for r in regions],
                value="Albany",
                clearable=False,
                style={'backgroundColor': 'white'},
                className="dropdown-select"
            )
        ], className="menu-item"),
        
        # Type filter
        html.Div([
            html.Div("AVOCADO TYPE", className="menu-title"),
            dcc.Dropdown(
                id="type-filter",
                options=[{"label": t.title(), "value": t} for t in avocado_types],
                value="conventional",
                clearable=False,
                style={'backgroundColor': 'white'},
                className="dropdown-select"
            )
        ], className="menu-item"),
        
        # Date range filter
        html.Div([
            html.Div("DATE RANGE", className="menu-title"),
            dcc.DatePickerRange(
                id="date-range",
                min_date_allowed=data["Date"].min().date(),
                max_date_allowed=data["Date"].max().date(),
                start_date=data["Date"].min().date(),
                end_date=data["Date"].max().date(),
                display_format='YYYY-MM-DD',
                className="date-picker"
            )
        ], className="menu-item")
    ], className="dash-menu"),
    
    # Main content area with charts
    html.Div([
        html.Div([
            dcc.Graph(id="price-chart"),
            dcc.Graph(id="volume-chart")
        ], className="chart-container")
    ], className="dash-content")
])

In [253]:
# Fourth cell - callback function (same as before)
@app.callback(
    [Output("price-chart", "figure"),
     Output("volume-chart", "figure")],
    [Input("region-filter", "value"),
     Input("type-filter", "value"),
     Input("date-range", "start_date"),
     Input("date-range", "end_date")]
)
def update_charts(region, avocado_type, start_date, end_date):
    # Filter the data
    filtered_data = data.query(
        "region == @region and type == @avocado_type "
        "and Date >= @start_date and Date <= @end_date"
    ).sort_values("Date")
    
    # Price chart
    price_chart = {
        "data": [{
            "x": filtered_data["Date"],
            "y": filtered_data["AveragePrice"],
            "type": "line",
            "name": "Average Price",
            "line": {"color": "#3498db", "width": 2.5},
            "hovertemplate": "$%{y:.2f}<extra></extra>"
        }],
        "layout": {
            "title": {
                "text": f"Average Price: {region} ({avocado_type.title()})",
                "font": {"size": 18}
            },
            "xaxis": {"title": "Date", "gridcolor": "#f0f0f0"},
            "yaxis": {
                "title": "Price ($)",
                "tickprefix": "$",
                "gridcolor": "#f0f0f0"
            },
            "plot_bgcolor": "white",
            "paper_bgcolor": "white",
            "hovermode": "x unified"
        }
    }
    
    # Volume chart
    volume_chart = {
        "data": [{
            "x": filtered_data["Date"],
            "y": filtered_data["Total Volume"],
            "type": "line",
            "name": "Total Volume",
            "line": {"color": "#e74c3c", "width": 2.5}
        }],
        "layout": {
            "title": {
                "text": f"Total Volume Sold: {region} ({avocado_type.title()})",
                "font": {"size": 18}
            },
            "xaxis": {"title": "Date", "gridcolor": "#f0f0f0"},
            "yaxis": {
                "title": "Volume",
                "gridcolor": "#f0f0f0"
            },
            "plot_bgcolor": "white",
            "paper_bgcolor": "white",
            "hovermode": "x unified"
        }
    }
    
    return price_chart, volume_chart

In [254]:
# Fifth cell - run the app
app.run_server(mode='inline', debug=True, height=800)